# Advent of Code 2018

This is a notebook of solutions for the [Advent of Code](http://adventofcode.com) for 2018. Each day will have a link to the problem description. Inputs are stored in the 'data' directory.

## Helper functions

In [3]:
import os
import urllib.request

#adapted from from norvig/pytudes repository
def Input(day, year=2018):
    "Open this day's input file."
    directory = 'data/advent{}/'.format(year)
    filename = directory+'day{}.txt'.format(day)
    try:
        return open(filename)
    except FileNotFoundError:
        if not os.path.exists(directory):
            os.makedirs(directory)

        urllib.request.urlretrieve("https://raw.githubusercontent.com/elahmo/advent-of-code/master/data/" + filename, filename)
        return Input(day)

def Inputstr(day, year=2018): 
    "The contents of this day's input file as a str."
    return Input(day, year).read().rstrip('\n')

def Inputlist(day, year=2018):
    lines = [line.rstrip('\n') for line in Input(day, year)]
    return lines

def mapt(fn, *args): 
    "Do a map, and make the results into a tuple."
    return tuple(map(fn, *args))

## [Day 1](https://adventofcode.com/2018/day/1): Chronal Calibration
Task is to find the resulting frequency after applying all the changes.

In [4]:
offsets = Inputlist(1)
total = sum([int(x) for x in offsets])

In [5]:
total

435

### Part Two
It seems the changes are repetitive, the task is to find the frequency that first appears for the second time.

In [6]:
frequency_list = {0} #store the reached frequencies
offsets = Inputlist(1) #parse the input
current_frequency = 0  #start at 0
looping = True
counter = 0
while looping:
    counter+=1
    for offset in offsets:
        current_frequency += int(offset)
        if current_frequency not in frequency_list:
            frequency_list.add(current_frequency)
        else:
            looping = False
            break
print('Final result:',current_frequency)

Final result: 245


**Comments**:
- **first part** was fairly easy, a simple sum in the list worked really well, using list compehensions for added beauty
- **second part** was a bit confusing at first, since the code seem to be stuck in an infinite loop
- the problem was that I used list in the beginning, which had quite slow lookup times and the large number of iterations took too long. Changing to set made things work super quick!

## [Day 2](https://adventofcode.com/2018/day/2): Inventory Management System
Task is to find the checksum, by multipling occurences of words that have two and three of any character only.

In [25]:
from collections import Counter

words = Inputlist(2)
#words = ['abcdef', 'bababc', 'abbcde', 'abcccd', 'aabcdd', 'abcdee', 'ababab'] #example scenario
two_letter = 0
three_letter = 0
for word in words:
    repetitions = set(v for k,v in Counter(word).items() if v == 2 or v == 3)
    if 2 in repetitions:
        two_letter += 1
    if 3 in repetitions:
        three_letter += 1
two_letter*three_letter

8296

### Part Two
Now, we should find the boxes that differ by fewest amount of characters, and find the common chars that appear in both.

In [55]:
from difflib import get_close_matches, SequenceMatcher
closest_pair = {'ratio':0, 'words':[]}
for word in words:
    match = SequenceMatcher()
    match.set_seqs(word, get_close_matches(word, words, 2)[1])
    if match.ratio() > closest_pair['ratio']:
        closest_pair['ratio'] = match.ratio()
        closest_pair['words'] = [word, get_close_matches(word, words, 2)[1]]

In [56]:
common_chars = ''
for a,b in zip(*closest_pair['words']):
    if a == b: common_chars+=a

In [57]:
common_chars

'pazvmqbftrbeosiecxlghkwud'

**Comments:**
- I feel this one is a bit harder than the first day, which makes sense, but the jump in complexity makes me fear the days to come!
- **first part** was fairly easy, thanks to the Counter library and smart usage of set which eliminates duplicates
- **second part** again was harder than the first, a bit confusing at first so I would definitely work on making instructions clearer
- I used the similar internal library to compare two words, I might try to rewrite this without any libraries but this was made specifically for this purpose and its relatively well performant
- I really liked the usage of *zip* and *args here to expand the elements when counting the common chars, this is some of the beauties of python

## [Day 3](https://adventofcode.com/2018/day/3): No Matter How You Slice It
Find squares that have been claimed by multiple Elves.

In [151]:
import re

claims = Inputlist(3)
claims_test = ['#1 @ 1,3: 4x4', '#2 @ 3,1: 4x4', '#3 @ 5,5: 2x2'] #test scenario, should give 4 as a result
cloth_map = [[0] * 1000 for i in range(1000)] #this could be improved to take the largest value in the input
for claim in claims:
    _, x, y, width, height = [int(s) for s in re.findall(r'-?\d+\.?\d*', claim)]
    for w in range(width):
        for h in range(height):
            cloth_map[y+h][x+w]+=1

In [152]:
from collections import Counter
from itertools import chain
count_claims = Counter(chain(*cloth_map))
count_claims

Counter({0: 648164,
         1: 234916,
         2: 88270,
         3: 21877,
         4: 5007,
         5: 1340,
         6: 368,
         7: 58})

In [153]:
print('Total:', sum([v for k,v in count_claims.items() if k >= 2]))

Total: 116920


### Part Two
A portion of cloth that has no claims at all should be found. Hmmm...

In [144]:
cloth_map = [[0] * 1000 for i in range(1000)]
individual_areas = {}
for claim in claims:
    cloth_id, x, y, width, height = [int(s) for s in re.findall(r'-?\d+\.?\d*', claim)]
    individual_areas[cloth_id] = width * height
    for w in range(width):
        for h in range(height):
            if cloth_map[y+h][x+w] == 0:
                cloth_map[y+h][x+w] = cloth_id
            else:
                cloth_map[y+h][x+w] = -1

In [146]:
from collections import Counter
from itertools import chain
count_claims = Counter(chain(*cloth_map))

In [150]:
for claim_id, area in count_claims.items():
    if area == individual_areas.get(claim_id):
        print('Bingo:', claim_id)

Bingo: 382


**Comments:**
- I am usually not a fan of 2D arrays, or list of lists. 
- This problem was fun, I did it easier than expect. I used my new learned knowledge to solve **part one**, by just incrementing how many claims are made for each tile, and the sum all tiles that have 2 or more claims.
- **part two** wasn't hard either. I first thought it might be expensive to do all of the things, since a title might be complete at the insertion time, but something might overwrite it later (most likely).
- A fairly simple solution is here, just comparing the individual area with the resulting area, again relying on the Counter from stdlib
- I am becoming a fan of 2D arrays!

## [Day 4](https://adventofcode.com/2018/day/4): Repose Record
Find which guard is sleeping the most, and what minute he most spent sleeping.

In [317]:
from datetime import datetime, timedelta
import re

schedule = Inputlist(4)
schedule.sort(key=lambda date: datetime.strptime(date[1:17], "%Y-%m-%d %H:%M"))
#schedule

In [297]:
def time_diff(guard_id, event):
    time_delta = ptime(event) - guards[guard_id]['time']
    return time_delta

def ptime(event):
    return datetime.strptime(event[1:17], "%Y-%m-%d %H:%M")

def sl_time(guard_id, event):
    return int(guards[guard_id]['time'].strftime('%M')), int(ptime(event).strftime('%M'))

guards = {}
for event in schedule:
    #go through each event
    #create a new guard, or update the current shift time
    if '#' in event:
        guard_id = re.findall('#(\d+)', event)[0]
        guards[guard_id] = guards.get(guard_id, {'sleeping_times':[]})
        guards[guard_id]['sleeping'] = guards[guard_id].get('sleeping', timedelta())
    else:
        #if taking a short nap, update time of nap
        if 'falls asleep' in event:
            guards[guard_id]['time'] = ptime(event)
        #if wakes up, calculate time slept
        if 'wakes up' in event:
            guards[guard_id]['sleeping'] += time_diff(guard_id, event)
            guards[guard_id]['sleeping_times'].append(sl_time(guard_id, event))

In [305]:
#guards

In [299]:
#find the guard with most minutes slept
sleepiest_guard = max(guards, key=lambda v: guards[v]['sleeping'])
sleepiest_guard

'73'

In [300]:
#find the minutes that are hit the most
minutes_during_shift = [0 for x in range(1,61)]
for a,b in guards[sleepiest_guard]['sleeping_times']:
    for i in range(a,b):
        minutes_during_shift[i] += 1
max_index, max_value = max(enumerate(minutes_during_shift), key=lambda p: p[1])
max_index, max_value

(44, 14)

### Part Two
Find wich guard most slept on a given minute in an hour.

In [312]:
#find the minutes that are hit the most, but for all guards
minutes_during_shift = {guard:[0 for x in range(1,61)] for guard in guards.keys()}
for guard,values in guards.items():
    for a, b in values['sleeping_times']:
        for i in range(a,b):
            minutes_during_shift[guard][i] += 1

In [316]:
max_index, max_guard, max_value = 0, None, 0
for guard, minutes in minutes_during_shift.items():
    max_index_guard, max_value_guard = max(enumerate(minutes), key=lambda p: p[1])
    if max_value_guard > max_value:
        max_index = max_index_guard
        max_value = max_value_guard
        max_guard = guard
max_index, max_guard, max_value

(26, '191', 17)

**Comments:**
- Well, this was super messy. I wont do these things anymore at 11pm.
- Overall, it doesnt seem that complex, but it is rather tedious with all datetime things.
- **part one** seemed quite OK, I just calculated the times someone was sleeping, and added those minutes without hour to another key in dictionary
- It took a bit of experimentation to get the max value from dict of dicts, but overall it was easy to get the sleepiest guard, and then for that guard just check his (or her) sleeping pattern and find which minutes were slept the most
- **part two** was far easier than in previous problems, at least it took far less time compared to the first part. I simply did the same thing as in the last stretch of part one when looking which minute was most slept, and generalised a bit and found needed guard and the minute